1. 准备原始文本数据
2. 将原始文本转换为bert相关输入格式
3. 在bert之上加入新layer成下游任务模型
4. 训练该下游任务模型
5. 对新样本做推测

In [11]:
import os
import pandas as pd

# 对数据进行清洗
df_train = pd.read_csv("data/train.csv")
empty_title = (df_train['title2_zh'].isnull() \
              | df_train['title1_zh'].isnull() \
              | (df_train['title2_zh'] == '') \
              | (df_train['title2_zh'] == '0'))
df_train = df_train[~empty_title]

print("原始样本数量：", len(df_train))

原始样本数量： 320543


In [12]:
# 剔除过长的样本数据避免 bert 无法将整个输入序列放入记忆力不强的 GPU
MAX_LENGTH = 30
df_train = df_train[~(df_train.title1_zh.apply(lambda x: len(x)) > MAX_LENGTH)]
df_train = df_train[~(df_train.title2_zh.apply(lambda x: len(x)) > MAX_LENGTH)]

# 只采用1%训练数据
SAMPLE_FRAC = 0.01
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9527)

# 去除不必要的单位并重新民命标题内容
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

# idempotence, 将结果另存为 tsv 供 PyTorch 使用
df_train.to_csv("train.tsv", sep="\t", index=False)

print("训练样本数量：", len(df_train))
df_train.head()

训练样本数量： 2657


,text_a,text_b,label
0,苏有朋要结婚了，但网友觉得他还是和林心如比较合适,好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！,unrelated
1,爆料李小璐要成前妻了贾乃亮模仿王宝强一步到位、快刀斩乱麻！,李小璐要变前妻了？贾乃亮可能效仿王宝强当机立断，快刀斩乱麻！,agreed
2,为彩礼，母亲把女儿嫁给陌生男子，十年后再见面，母亲湿了眼眶,阿姨，不要彩礼是觉得你家穷，给你台阶下，不要以为我嫁不出去！,unrelated
3,猪油是个宝，一勺猪油等于十副药，先备起来再说,传承千百的猪油为何变得人人唯恐避之不及？揭开猪油的四大谣言！,unrelated
4,剖析：香椿，为什么会致癌？,香椿含亚硝酸盐多吃会致癌？测完发现是谣言,disagreed


由于样本数据中的unrelated 的数量为68% ,因此我们用的bert训练出的最少要超过68% 的baseline才可以：

In [13]:
df_train.label.value_counts() / len(df_train)

unrelated    0.679338
agreed       0.294317
disagreed    0.026346
Name: label, dtype: float64

对测试数据进行处理，满足格式要求

In [14]:
df_test = pd.read_csv("data/test.csv")
df_test = df_test.loc[:, ["title1_zh", "title2_zh", "id"]]
df_test.columns = ["text_a", "text_b", "Id"]
df_test.to_csv("test.tsv", sep="\t", index=False)

print("預測樣本數：", len(df_test))
df_test.head()

預測樣本數： 80126


,text_a,text_b,Id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191


In [15]:
ratio = len(df_test) / len(df_train)
print("测试数据样本树 / 训练数据样本书 = {:.1f} 倍".format(ratio))

测试数据样本树 / 训练数据样本书 = 30.2 倍


将数据内容转化为 dataset 相容的格式内容

In [19]:
"""
作为一个可以用来读取训练 / 测试集的 Dataset, 这是你需要彻底了解的部分，
此 Dataset 每次將 tsv 里的一组成对句子转化成 BERT 相容的格式，并回传3哥 tensors：
- tokens_tensor：两个句子合并后的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用来识别两个句子的界限 binary tensor
- label_tensor：将分类目标转化为类别索引 tensor, 如果是测试集则回传 None
"""
from torch.utils.data import Dataset
import torch

class FakeNewsDataset(Dataset):
    # 读取之前初始化后的 tsv 并初始化一部分参数
    def __init__(self, mode, tokenizer):
        # 一般训练会需要 dev set
        assert mode in ["train", "test"]
        self.mode = mode
        # 大数据你会需要用 iterator = True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'agreed': 0, 'disagreed': 1, 'unrelated': 2}
        self.tokenizer = tokenizer

    # 定义回传一些训练 / 测试数据函数
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, test_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)

        # 建立第一个句子的 bert tokens 并加入分割符号 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        print(tokens_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)

        # 第二个句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a

        # 第二个句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
